In [1]:
import os
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# load files

In [2]:
def load_file(
    path="", 
    usecols=None
):
    # LOAD DATAFRAME
    if usecols is not None: 
        df = pd.read_parquet(path, columns=usecols)
    else: 
        df = pd.read_parquet(path)
    
    # REDUCE DTYPE FOR CUSTOMER AND DATE
    df["customer_ID"] = df["customer_ID"].str[-16:]
    
    hex_to_int = lambda x: int(x, 16)
    df[["customer_ID"]] = df[["customer_ID"]].applymap(lambda x: int(x, 16))
    
    df["customer_ID"] = df["customer_ID"].astype("int64")
    df["S_2"] = pd.to_datetime(df["S_2"])
    
    # SORT BY CUSTOMER AND DATE (so agg("last") works correctly)
    df = df.sort_values(["customer_ID", "S_2"])
    df = df.reset_index(drop=True)
    
    # FILL NAN
    print("shape of data:", df.shape)
    
    return df

In [3]:
print("Reading data...")
TRAIN_PATH = "../input/amex-data-integer-dtypes-parquet-format/train.parquet"
train = load_file(path = TRAIN_PATH)

TEST_PATH = "../input/amex-data-integer-dtypes-parquet-format/test.parquet"
test = load_file(path = TEST_PATH)

Reading data...
shape of data: (5531451, 190)
shape of data: (11363762, 190)


In [4]:
# revert to nan
train[train==-1] = np.nan
test[test==-1] = np.nan

In [5]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,-9223358381327749917,2017-03-31,0.342033,9,0.298571,0.028331,0.506896,0.793958,0.0,0.823765,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.004787,0.0
1,-9223358381327749917,2017-04-07,0.340178,16,0.353684,0.026975,0.505335,0.795727,0.0,0.825231,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.003442,0.0
2,-9223358381327749917,2017-05-23,0.356010,1,0.448582,0.026601,0.506290,0.530133,0.0,0.923707,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.003340,0.0
3,-9223358381327749917,2017-06-22,0.378665,1,0.443752,0.024322,0.509069,0.539285,0.0,0.915724,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.007556,0.0
4,-9223358381327749917,2017-07-22,0.416543,1,0.463824,0.023064,0.505335,0.461935,0.0,0.919373,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.005299,0.0


In [6]:
train.shape

(5531451, 190)

# add number of observations

In [7]:
def add_observation(df):
    
    df["number_of_observations"] = df.groupby("customer_ID")["customer_ID"].transform("count")
    
    return df

In [8]:
train = add_observation(train)
test = add_observation(test)

# add first occurance flag

In [9]:
def add_first_occurance(df):
    
    df["index"] = df.index.tolist()
    first_occurance_index = df[["customer_ID", "index"]].groupby("customer_ID").first()["index"].tolist()
    
    df["first_occurance"] = 0
    df.loc[df["index"].isin(first_occurance_index), "first_occurance"] = 1
    
    df = df.drop(["index"], axis=1)
    
    return df

In [10]:
train = add_first_occurance(train)
test = add_first_occurance(test)

# process nan

In [11]:
# get nan clusters first
cols = sorted(train.columns[2:].tolist())
nas = train[cols].isna().sum(axis=0).reset_index(name="NA_count")
nas["group_count"] = nas.loc[nas.NA_count > 0].groupby("NA_count").transform("count")
clusters = nas.loc[nas.group_count > 10].sort_values(["NA_count","index"]).groupby("NA_count")["index"].apply(list).values

In [12]:
def process_type_0_nan(df, cluster):
    
    df["type_0_nan"] = 0
    df.loc[df[cluster[0]].isnull(), "type_0_nan"] = 1
    df.loc[df["type_0_nan"] == 1, cluster] = df.loc[df["type_0_nan"] == 1, cluster].fillna(0)
    
    return df

In [13]:
train = process_type_0_nan(train, clusters[0])
test = process_type_0_nan(test, clusters[0])

In [14]:
def process_type_1_nan(df, cluster):
    
    df["type_1_nan"] = 0
    df.loc[
        (df[cluster[0]].isnull()) & (df["first_occurance"] == 1), 
        "type_1_nan"
    ] = 1
    df.loc[
        (df[cluster[0]].isnull()) & (df["first_occurance"] == 0), 
        "type_1_nan"
    ] = 2
    
    # fill type_1_nan == 1 by 0
    df.loc[df["type_1_nan"] == 1, cluster] = df.loc[df["type_1_nan"] == 1, cluster].fillna(0)
    
    # fill type_1_nan == 0 by mean of t - 1 and t + 1
    ffill = df[["customer_ID", "type_1_nan"] + cluster].copy()
    bfill = df[["customer_ID", "type_1_nan"] + cluster].copy()
    
    ffill[cluster] = ffill[cluster].fillna(method="ffill")
    bfill[cluster] = bfill[cluster].fillna(method="bfill")
    
    df.loc[df["type_1_nan"] == 2, cluster] = (ffill.loc[ffill["type_1_nan"] == 2, cluster] + \
                                              bfill.loc[ffill["type_1_nan"] == 2, cluster]) / 2
    
    return df

In [15]:
train = process_type_1_nan(train, clusters[1])
test = process_type_1_nan(test, clusters[1])

In [16]:
def process_type_2_nan(df, cluster):
    
    df["type_2_nan"] = 0
    df.loc[
        (df[cluster[0]].isnull()) & (df["first_occurance"] == 1), 
        "type_2_nan"
    ] = 1
    df.loc[
        (df[cluster[0]].isnull()) & (df["first_occurance"] == 0), 
        "type_2_nan"
    ] = 2
    
    return df

In [17]:
train = process_type_2_nan(train, clusters[2])
test = process_type_2_nan(test, clusters[2])

# add time id

In [18]:
def add_time_id(df):
    
    df["time_id"] = df.groupby(["customer_ID"]).cumcount()
    
    return df

In [19]:
train = add_time_id(train)
test = add_time_id(test)

# feature engineering

In [20]:
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(["customer_ID"])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + "_diff1" for col in df[num_features].columns])
    df1["customer_ID"] = customer_ids
    return df1


def process_and_feature_engineer(df):

    all_cols = [c for c in list(df.columns) if c not in ["customer_ID", "S_2", "first_occurance", "time_id"]]
    nan_related_features = [
        "number_of_observations",
        "type_0_nan",
        "type_1_nan",
        "type_2_nan"
    ]
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    num_features = [col for col in all_cols if col not in (cat_features + nan_related_features)]
    
    print("num features")
    df_num_agg = df.groupby("customer_ID")[num_features].agg(["first", "mean", "std", "min", "max", "last"])
    df_num_agg.columns = ["_".join(x) for x in df_num_agg.columns]
    df_num_agg.reset_index(inplace = True)

    print("lag features")
    for col in df_num_agg:
        for col_2 in ["first", "mean", "std", "min", "max"]:
            if "last" in col and col.replace("last", col_2) in df_num_agg:
                df_num_agg[col + "_lag_sub"] = df_num_agg[col] - df_num_agg[col.replace("last", col_2)]
                df_num_agg[col + "_lag_div"] = df_num_agg[col] / df_num_agg[col.replace("last", col_2)]
    print("num + lag features shape:", df_num_agg.shape)
    
    print("cat features")
    df_cat_agg = df.groupby("customer_ID")[cat_features].agg(["count", "first", "last", "nunique"])
    df_cat_agg.columns = ["_".join(x) for x in df_cat_agg.columns]
    df_cat_agg.reset_index(inplace = True)
    print("cat features shape:", df_cat_agg.shape)
        
    print("diff features")
    df_diff = get_difference(df, num_features)
    df = df_num_agg.merge(df_cat_agg, how = "inner", on = "customer_ID").merge(df_diff, how = "inner", on = "customer_ID")
    print("all features shape:", df.shape)
    
    return df

In [21]:
train = process_and_feature_engineer(train)
test = process_and_feature_engineer(test)

num features
lag features
num + lag features shape: (458913, 1417)
cat features
cat features shape: (458913, 45)
diff features


100%|██████████| 458913/458913 [10:34<00:00, 722.81it/s]


all features shape: (458913, 1638)
num features
lag features
num + lag features shape: (924621, 1417)
cat features
cat features shape: (924621, 45)
diff features


100%|██████████| 924621/924621 [21:22<00:00, 720.90it/s]


all features shape: (924621, 1638)


In [22]:
train.isnull().sum()

customer_ID         0
P_2_first        2434
P_2_mean         2434
P_2_std          7829
P_2_min          2434
                ...  
D_141_diff1      5120
D_142_diff1    380530
D_143_diff1      5120
D_144_diff1      5209
D_145_diff1      5120
Length: 1638, dtype: int64

In [23]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,...,D_136_diff1,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1
0,-9223358381327749917,0.342033,0.415868,0.057145,0.340178,0.498727,0.387708,9,2.615385,4.628507,...,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,0.006017,0.0
1,-9223193039457028513,0.979310,0.974068,0.013094,0.964483,1.002478,1.001372,0,0.000000,0.000000,...,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,-0.001580,0.0
2,-9223189665817919541,0.812751,0.802447,0.038025,0.694073,0.828761,0.694073,0,0.000000,0.000000,...,NaN,NaN,NaN,-1.0,0.0,-1.020667,NaN,-1.0,-1.331635,-1.0
3,-9223188534444851899,0.794211,0.791203,0.002688,0.786647,0.794826,0.787945,0,0.000000,0.000000,...,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,-0.002107,0.0
4,-9223173911659837606,0.252421,0.115666,0.078554,0.038207,0.252421,0.040486,0,4.384615,6.144625,...,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,0.000790,0.0


# Add target

In [41]:
def add_target(df):
    
    targets = pd.read_csv("../input/train_labels.csv")
    
    # REDUCE DTYPE FOR CUSTOMER AND DATE
    targets["customer_ID"] = targets["customer_ID"].str[-16:]
    
    hex_to_int = lambda x: int(x, 16)
    targets[["customer_ID"]] = targets[["customer_ID"]].applymap(lambda x: int(x, 16))
    targets["customer_ID"] = targets["customer_ID"].astype("int64")
    
    targets = targets.set_index("customer_ID")
    df = df.set_index("customer_ID")
    
    df = df.merge(targets, left_index=True, right_index=True, how="left")
    df.target = df.target.astype("int8")

    # NEEDED TO MAKE CV DETERMINISTIC (cudf merge above randomly shuffles rows)
    df = df.sort_index().reset_index()
    
    return df

In [42]:
train = add_target(train)

In [43]:
train.shape

(458913, 1639)

In [44]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,...,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1,target
0,-9223358381327749917,0.342033,0.415868,0.057145,0.340178,0.498727,0.387708,9,2.615385,4.628507,...,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,0.006017,0.0,1
1,-9223193039457028513,0.979310,0.974068,0.013094,0.964483,1.002478,1.001372,0,0.000000,0.000000,...,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,-0.001580,0.0,0
2,-9223189665817919541,0.812751,0.802447,0.038025,0.694073,0.828761,0.694073,0,0.000000,0.000000,...,NaN,NaN,-1.0,0.0,-1.020667,NaN,-1.0,-1.331635,-1.0,0
3,-9223188534444851899,0.794211,0.791203,0.002688,0.786647,0.794826,0.787945,0,0.000000,0.000000,...,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,-0.002107,0.0,0
4,-9223173911659837606,0.252421,0.115666,0.078554,0.038207,0.252421,0.040486,0,4.384615,6.144625,...,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,0.000790,0.0,1


# label encoding

In [45]:
def label_encoding(df):
    
    cat_features_base = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ] 
    cat_features = [
        "{}_last".format(feature) for feature in cat_features_base
    ]
    
    for feature in cat_features:
        encoder = LabelEncoder()
        df[feature] = encoder.fit_transform(df[feature])
    
    return df

In [46]:
train = label_encoding(train)
test = label_encoding(test)

# save files

In [47]:
train.to_parquet("../input/train_public.parquet")
test.to_parquet("../input/test_public.parquet")